In [3]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler  # for streaming response
from langchain.callbacks.manager import CallbackManager

In [4]:
from langchain import HuggingFacePipeline, PromptTemplate

In [5]:
import torch
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, TextStreamer, pipeline , GenerationConfig
from deep_translator import GoogleTranslator
from auto_gptq import AutoGPTQForCausalLM
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import GPT4All
from deep_translator import GoogleTranslator

In [7]:
from llama_cpp import ChatCompletionMessage
from deep_translator import GoogleTranslator
import asyncio
import sys
import json
import os
import pickle

# Loading Embeddings

In [8]:
embeddings = HuggingFaceInstructEmbeddings(
        model_name="intfloat/multilingual-e5-large",
        model_kwargs={"device": DEVICE},
    )

load INSTRUCTOR_Transformer
max_seq_length  512


# Loading Model (Using Langchain.GPT4All)

In [7]:
llm = GPT4All(
    device = 'gpu',
    model="C:/Users/Alaa AI/Python Projects/Chatbots/GPT by me/alaa_ai_model_mistral_v1.9.gguf",
    max_tokens=2048,
    allow_download=False,
    backend="mistral",
    verbose=False,
)

2496563451600


# Initializing Knowledge Base -- Running only for first time or When updating resourcing 

In [8]:
knowledge_base_loader = PyPDFDirectoryLoader("pdfs")
knowledge_base = knowledge_base_loader.load()

In [9]:
knowledge_base_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
knowledge_base_texts = knowledge_base_text_splitter.split_documents(knowledge_base)
len(knowledge_base_texts)

24941

In [13]:
# %%time
# updated_db_knowledge_base = Chroma.from_documents(knowledge_base_texts, embeddings, persist_directory="db-knowledge-base")

# Load existed knowledge base 

In [10]:
db_knowledge_base = Chroma(persist_directory="db-knowledge-base", embedding_function=embeddings)

# Initialize Patient Case Data and Updating Knowledge base -- Continue with us 

In [ ]:
input_loader = PyPDFDirectoryLoader("input")
input_docs = input_loader.load()
len(input_docs)

In [11]:
input_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
input_texts = input_text_splitter.split_documents(input_docs)
len(input_texts)

daily lives and brings them to life in a user-friendly format. The authors discuss many 
different settings in which ADHD may cause difficulties, including work, school, matters 
of physical health and well-being, and the issue of excessive use of technology. Although 
written for consumers, clinicians will find the book to be a clinically useful tool for their 
adult patients with ADHD, serving as a companion to the newly updated and expanded


In [13]:
%%time
updated_db_knowledge_base = Chroma.from_documents(input_texts, embeddings, persist_directory="db-knowledge-base")

# Load if the patient is already existed and need more Details about him according to the provided resources 

In [9]:
updated_db_knowledge_base = Chroma(persist_directory="db-knowledge-base", embedding_function=embeddings)

# VectorStore not used because I've used Chroma

In [16]:
# vectorstore = FAISS.from_documents(texts, embeddings)

# Initialize System Prompt

In [12]:
# system_prompt = """You are a Definition Rate System . Use the following pieces of context to Give the rate only at the end. If you don't know the answer, just say that you don't know, and try to make up an answer."""
# system_prompt = """You are a Definition Rate System, you will use the provided context to rate user Definitions.
# Read the given context before answering questions and think step by step. If you can not answer a user question based on 
# the provided context, inform the user. Do not use any other information for answering user. Provide the rate only to the question."""
system_prompt = """You are a helpful assistant, you will use the provided context to answer user questions.
Read the given context before answering questions and think step by step. If you can not answer a user question based on 
the provided context, inform the user. Do not use any other information for answering user. Provide a detailed answer to the question."""

In [10]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [11]:
# SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, and try to make up an answer."
# SYSTEM_PROMPT = "Use the following pieces of context to Give me a report about the following case of patient : which contains Patient Details , Diagnostic Scale Results , Executive Function Scale Results , Learning Disability Assessment , Interview and case History Insights , perliminary Diagnosis , Proposed Intervention program , Medication Suggestions , Professional Review Notes. If you don't know the answer, just say that you don't know, and try to make up an answer."

# SYSTEM_PROMPT = "Explain [medical condition] in simple language for a newly diagnosed patient. Give me 3 examples. You have 200 words max for each explanation"
template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

# Adding the Question

In [12]:
question = "Give me an organized Detailed Form about the Patient Details provided for examinee 'o'"
input_text = GoogleTranslator(source = 'auto' , target = 'en').translate(question)

In [13]:
print(input_text)

Give me an organized Detailed Form about the Patient Details provided for examinee 'o'


# Initialize the Prompt Template

In [14]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Use vectorstore if you use it 

In [16]:
# retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
# docs = retriever.get_relevant_documents(input_text)

# Initialize The Chain

In [17]:
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=updated_db_knowledge_base.as_retriever(search_kwargs={"k": 15}),
#     return_source_documents=True,
#     chain_type_kwargs={"prompt": prompt},
#     callbacks=callback_manager,
# )

In [18]:
qa_chain = RetrievalQA.from_chain_type(
  llm, retriever=updated_db_knowledge_base.as_retriever(search_kwargs={"k": 3})
    ,chain_type= "stuff", chain_type_kwargs={"prompt": prompt},
)
# qa_chain(question)

ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [61]:
%%time
res = qa_chain(return_only_outputs = True , 
   inputs = input_text
)

CPU times: total: 1min 49s
Wall time: 43.7 s


In [62]:
res["result"]

"\n\n[INST] <>\nPatient Name: Examinee 'o'\nDate of Birth: Not Provided\nGender: Not Provided\nHeight: Not Provided\nWeight: Not Provided\nBlood Pressure: Not Provided\nTemperature: Not Provided\nPulse Rate: Not Provided\nRespiratory Rate: Not Provided\nAllergies: Not Provided\nMedications: Not Provided\nDiagnosis: Not Provided\nSymptoms: Not Provided\nFamily History: Not Provided\nPhysical Examination Findings: Not Provided\nLab Results: Not Provided\nImaging Studies: Not Provided\nProcedures Performed: Not Provided\nConsultations: Not Provided\nReferrals: Not Provided\nAdmission Date: Not Provided\nDischarge Date: Not Provided\nLength of Stay: Not Provided\nReason for Admission: Not Provided\nReason for Discharge: Not Provided\nTreatment Plan: Not Provided\nFollow-up Appointments: Not Provided\n[/INST]"

In [30]:
result = res["result"].split(' ')
print(result)

['?']


In [163]:
import re
rate = re.findall(r'\b\d+\b', res["result"])

In [164]:
rate[0]

'2'

In [134]:
print(GoogleTranslator(source = 'auto' , target = 'ar').translate(res["result"]))

</s>
<s>[INST] أنت نظام معدل تعريف، وسوف تستخدم السياق المتوفر لتقييم تعريفات المستخدم.
اقرأ السياق المحدد قبل الإجابة على الأسئلة وفكر خطوة بخطوة. إذا لم تتمكن من الإجابة على سؤال المستخدم على أساس
السياق المقدم، إبلاغ المستخدم. لا تستخدم أي معلومات أخرى للرد على المستخدم. توفير المعدل فقط على السؤال.

السياق: الموضوع: الحجاب
تعريف الموضوع: عند وفاته
المعدل: 0
التقييم: صفر

الموضوع: الحجاب
تعريف الموضوع: قصير
المعدل: 1
التقييم: واحد

الموضوع: الحجاب
تعريف الموضوع: ملون
المعدل: 2
التقييم: اثنان
            المستخدم: ما هو معدل "الحجاب" عندما يتم تعريفه على أنه "عند وفاته"؟ [/إنست]</s>
<s>[INST] أنت نظام معدل تعريف، وسوف تستخدم السياق المتوفر لتقييم تعريفات المستخدم.
اقرأ السياق المحدد قبل الإجابة على الأسئلة وفكر خطوة بخطوة. إذا لم تتمكن من الإجابة على سؤال المستخدم على أساس
السياق المقدم، إبلاغ المستخدم. لا تستخدم أي معلومات أخرى للرد على المستخدم


In [25]:
qa_chain(question)

{'query': "ممكن تكلمنى عن قانون تعليم الأطفال idea 'Respond in details'",
 'result': "\n</s>\n\nThe Education for Individuals with Difficulties Act 2004, also known as the Improvement of Education for Disabilities with Individuals Act, is a law that focuses on providing education to individuals with difficulties or disabilities. This act covers developmental delay and severe and multiple disabilities. The purpose of this law is to ensure that these children receive an appropriate education tailored to their needs.\n\nThe law applies from birth up to the age of 15, and in cases of a severe delay, it may be applied to larger loans. It takes into account that the child's subculture might not encourage long verbal responses, so the focus is on providing an education that caters to their specific needs and abilities."}

In [ ]:
qa_chain("what is the figure in the document? Extract the answer from the text")